## Learning SMACH


In [16]:
get_ipython().system_raw('roslaunch locobot_control main.launch use_arm:=true use_base:=true & ')

### Creating a State Machine

To create a Smach state machine, you first create a number of states, and then add those states to a State Machine container.

In [17]:
#!/usr/bin/env python

import rospy
import smach
import smach_ros
import time
from pyrobot import Robot

base_config_dict={'base_controller': 'ilqr'} 
robot = Robot('locobot', base_config=base_config_dict)

### Creating a state

In [18]:
# define state Base_move
class Base_move(smach.State):
    def __init__(self):
        smach.State.__init__(self, outcomes=['forward','backward'])
        self.counter = 0
        forward_position = [1,0.0,0.0]
        backward_position = [0.0,0.0,0.0]

    def execute(self, userdata):
        rospy.loginfo('Executing state Base_move')
        if self.counter < 2:
            self.counter += 1
            
            robot.base.go_to_absolute(forward_position)
            time.sleep(1)
            return 'forward'
        else:
            robot.base.go_to_absolute(backward_position)
            time.sleep(1)
            return 'backward'

In [19]:
# define state Arm_move
class Arm_move(smach.State):
    def __init__(self):
        smach.State.__init__(self, outcomes=['home'])

    def execute(self, userdata):
        rospy.loginfo('Executing state Arm_move')
        robot.arm.go_home()
        time.sleep(1)
        return 'home'

### Adding states to a state machine

In [20]:
# main
def main():
    rospy.init_node('smach_pyrobot')

    # Create a SMACH state machine
    sm = smach.StateMachine(outcomes=['End'])

    # Open the container
    with sm:
        # Add states to the container
        smach.StateMachine.add('Base_move', Base_move(), transitions={'forward':'Arm_move', 'backward':'End'})
        smach.StateMachine.add('Arm_move', Arm_move(), transitions={'home':'Base_move'})

    # Create and start the introspection server
    sis = smach_ros.IntrospectionServer('my_smach_introspection_server', sm, '/SM_ROOT')
    sis.start()

    # Execute SMACH plan
    outcome = sm.execute()
    
    # Wait for ctrl-c to stop the application
    rospy.spin()
    sis.stop()

if __name__ == '__main__':
    main()

ROSException: rospy.init_node() has already been called with different arguments: ('smach_example_state_machine', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/jacky/.local/share/jupyter/runtime/kernel-5af2d701-4dea-47c7-9be2-1590b2836798.json'], False, None, False, False)